In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys

sys.path.append("/root/workspace/seed")

In [4]:
from seed.pipeline.pipeline import Pipeline

In [5]:
from seed.retriever import Retriever

retriever = Retriever.get("dpr")

Attempting to initialize pre-built index wikipedia-dpr.
/root/.cache/pyserini/indexes/index-wikipedia-dpr-20210120-d1b9e6.c28f3a56b2dfcef25bf3bf755c264d04 already exists, skipping download.
Initializing wikipedia-dpr...


Some weights of the model checkpoint at facebook/dpr-question_encoder-multiset-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Attempting to initialize pre-built index wikipedia-dpr-multi-bf.
/root/.cache/pyserini/indexes/dindex-wikipedia-dpr_multi-bf-20200127-f403c3.29eb39fe0b00a03c36c0eeae4c24f775 already exists, skipping download.
Initializing wikipedia-dpr-multi-bf...
Attempting to initialize pre-built index wikipedia-dpr.
/root/.cache/pyserini/indexes/index-wikipedia-dpr-20210120-d1b9e6.c28f3a56b2dfcef25bf3bf755c264d04 already exists, skipping download.
Initializing wikipedia-dpr...


In [11]:
from seed.verifier import Verifier

verifier = Verifier.get("tapex")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


In [12]:
from seed.extractor import Extractor

extractor = Extractor.get("unifiedqa")

2022-06-30 21:51:59,639 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4


/opt/conda/envs/ml/lib/python3.8/site-packages/huggingface_hub/file_download.py:562: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


2022-06-30 21:52:05,419 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [13]:
pipeline = Pipeline(retriever, verifier, extractor)

In [14]:
import pandas as pd
import jsonlines

df = pd.DataFrame(list(jsonlines.open("../data/totto/totto_dev_data_augmented.jsonl")))

In [17]:
import json

for idx, obj in df.iterrows():
    if not obj["highlighted_cells"]:
        continue
    table = pd.DataFrame(json.loads(obj["table"]))
    rows = [x[0] for x in obj["highlighted_cells"]]
    most_dominant_row = max(set(rows), key=rows.count)
    cols = [x[1] for x in obj["highlighted_cells"] if x[0] == most_dominant_row]
    table = table.iloc[[most_dominant_row], ].reset_index().drop("index", axis=1)
    print(table)

    result = pipeline.run(table)
    break

  Place            Player   Country      Score
0     1  Willie Park, Jr.  Scotland  76-75=151
Player is Willie Park, Jr. and Country is Scotland
DenseSearchResult(docid='15216092', score=82.69419984817505)
DenseSearchResult(docid='4714673', score=82.59816961288452)
DenseSearchResult(docid='7757710', score=82.01664094924926)
DenseSearchResult(docid='4714664', score=81.66472749710083)
DenseSearchResult(docid='7698329', score=81.56730012893676)
DenseSearchResult(docid='8665568', score=81.31617097854614)
DenseSearchResult(docid='7757717', score=81.27319459915161)
DenseSearchResult(docid='4714662', score=81.12369766235352)
DenseSearchResult(docid='4636076', score=81.00425395965576)
DenseSearchResult(docid='12438200', score=80.87616395950317)
2013–14 BeNe League The 2013–14 season of the Women's BeNe League is the second season of the Belgium and Netherlands' women's football top level league.   Place            Player   Country      Score
0     1  Willie Park, Jr.  Scotland  76-75=151
The d

AttributeError: 'list' object has no attribute 'lower'

In [38]:
result = retriever.searcher.search("Player is Willie Park Jr. and country is Scotland", k=20)

In [37]:
for doc in result:
    print(retriever.ssearcher.doc(int(doc.docid)).raw())

{
  "id" : "17326486",
  "contents" : "\"2013–14 BeNe League\"\n2013–14 BeNe League The 2013–14 season of the Women's BeNe League is the second season of the Belgium and Netherlands' women's football top level league. The defending champion is FC Twente. The season started on 30 August 2013 and is played in a single division contrary to last season. Originally planned as a 16 team league, one team withdrew before the season and another one withdrew during the season. Thus 14 teams made the final standings. With six matches to spare Standard Liège already was set as the best placed Belgian side. The league was won by Twente for the"
}
{
  "id" : "103002",
  "contents" : "\"Definable real number\"\nnumbers including π and \"\"e\"\". Like the algebraic numbers, the computable numbers also form a subfield of the real numbers, and the positive computable numbers are closed under taking \"\"n\"\"th roots for each positive \"\"n\"\". Not all real numbers are computable. The entire set of comp

In [31]:
retriever.ssearcher.doc(int(4714673)).raw()

'{\n  "id" : "103002",\n  "contents" : "\\"Definable real number\\"\\nnumbers including π and \\"\\"e\\"\\". Like the algebraic numbers, the computable numbers also form a subfield of the real numbers, and the positive computable numbers are closed under taking \\"\\"n\\"\\"th roots for each positive \\"\\"n\\"\\". Not all real numbers are computable. The entire set of computable numbers is countable, so most reals are not computable. Specific examples of noncomputable real numbers include the limits of Specker sequences, and algorithmically random real numbers such as Chaitin\'s Ω numbers. Another notion of definability comes from the formal theories of arithmetic, such as Peano arithmetic. The language of arithmetic has symbols for 0,"\n}'